**Ratchanon Tarawan 65070503464**

# Homework 2: Introducing Classification

Objectives:
- To gain hands-on experience classifying small dataset
- To implement concepts related to Decision Tree classifier (i.e. Entropy, Information Gain), along with the Decision Tree algorithm

In [70]:
import pandas as pd
import numpy as np

# Read the data
df = pd.read_csv('toy_data.csv')
df

,age,income,student,credit rating,buys computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31-40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31-40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [71]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            14 non-null     object
 1   income         14 non-null     object
 2   student        14 non-null     object
 3   credit rating  14 non-null     object
 4   buys computer  14 non-null     object
dtypes: object(5)
memory usage: 688.0+ bytes
None


Compute Information Gain based on student attribute

In [72]:
target = df['buys computer'].value_counts()
# student= df['student'].value_counts()
tar_yes = target.yes
tar_no = target.no
tar_all = tar_yes + tar_no

In [73]:
Target = -(tar_yes/tar_all)*np.log2(tar_yes/tar_all) - (tar_no/tar_all)*np.log2(tar_no/tar_all)
print(Target)

0.9402859586706311


In [74]:
student = df['student'].value_counts()
student_no= student.no
student_yes= student.yes

yes_count = df[df['student'] == 'yes']['buys computer'].value_counts()
no_count = df[df['student'] == 'no']['buys computer'].value_counts()

print(yes_count, no_count)
#print(high_count)

buys computer
yes    6
no     1
Name: count, dtype: int64 buys computer
no     4
yes    3
Name: count, dtype: int64


In [75]:
yes = -(yes_count['yes']/student_yes)*np.log2(yes_count['yes']/student_yes) - (yes_count['no']/student_yes)*np.log2(yes_count['no']/student_yes)
no = -(no_count['yes']/student_no)*np.log2(no_count['yes']/student_no) - (no_count['no']/student_no)*np.log2(no_count['no']/student_no)

In [76]:
entro_student = (student_yes/tar_all)*yes + (student_no/tar_all)*no
entro_student

0.7884504573082896

In [77]:
gain = Target - entro_student

In [78]:
print('Gain (Student)' , gain)


Gain (Student) 0.15183550136234159


Compute Information Gain based on credit rating attribute

In [79]:
target = df['buys computer'].value_counts()
# credit_rating  = df['credit rating'].value_counts()
tar_yes = target.yes
tar_no = target.no
tar_all = tar_yes + tar_no

In [80]:
Target = -(tar_yes/tar_all)*np.log2(tar_yes/tar_all) - (tar_no/tar_all)*np.log2(tar_no/tar_all)
print(Target)

0.9402859586706311


In [81]:
credit_rating  = df['credit rating'].value_counts()
credit_rating_excellent= credit_rating.excellent
credit_rating_fair= credit_rating.fair

excellent_count = df[df['credit rating'] == 'excellent']['buys computer'].value_counts()
fiar_count = df[df['credit rating'] == 'fair']['buys computer'].value_counts()

print(excellent_count, fiar_count)
#print(high_count)

buys computer
no     3
yes    3
Name: count, dtype: int64 buys computer
yes    6
no     2
Name: count, dtype: int64


In [82]:
excellent = -(excellent_count['yes']/credit_rating_excellent)*np.log2(excellent_count['yes']/credit_rating_excellent) - (excellent_count['no']/credit_rating_excellent)*np.log2(excellent_count['no']/credit_rating_excellent)
fair = -(fiar_count['yes']/credit_rating_fair)*np.log2(fiar_count['yes']/credit_rating_fair) - (fiar_count['no']/credit_rating_fair)*np.log2(fiar_count['no']/credit_rating_fair)


In [83]:
entro_credit_rating = (credit_rating_excellent/tar_all)*excellent + (credit_rating_fair/tar_all)*fair

In [84]:
gain = Target - entro_credit_rating
print('Gain (Credit Rating)' , gain)

Gain (Credit Rating) 0.04812703040826949


Construct a decision tree using the algorithm in the slides

In [85]:
import numpy as np
from collections import Counter

In [86]:
import numpy as np
from collections import Counter

# Count entropy
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

# Create node
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

# Create a Decision algorithm model
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common


In [87]:
# Given mappings
mapping = {'yes': 1, 'no': 0}
mapping2 = {'high': 2, 'medium': 1, 'low': 0}
mapping3 = {'fair': 1, 'excellent': 0}
mapping4 = {'<=30': 0, '31-40': 1, '>40': 2}

# Applying mappings
df['buys computer'] = df['buys computer'].map(mapping)
df['income'] = df['income'].map(mapping2)
df['credit rating'] = df['credit rating'].map(mapping3)
df['age'] = df['age'].map(mapping4)
df['student'] = df['student'].map(mapping)

# Separate features
X_train = df.drop('buys computer', axis=1).values
y_train = df['buys computer'].values

# Fit the decision tree on the training data
tree = DecisionTree(min_samples_split=2, max_depth=4, n_feats=None)
tree.fit(X_train, y_train)

# Test data
X_test = np.array([
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [2, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 1]
])

# Make predictions on the test data
predictions = tree.predict(X_test)
result_list = []

# Print the predictions
print("Predictions:")
for i in predictions:
    if i == 1:
        result_list.append('buy')
    else:
        result_list.append('not buy')
print(result_list)


Predictions:
['not buy', 'buy', 'buy', 'buy', 'not buy']
